In [93]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in

from datetime import datetime, timedelta

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# importing libraries

import csv
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow import keras
from keras import callbacks
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping
import math
from sklearn.metrics import mean_squared_error


import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [94]:
selected_columns = [
    # 'Street Code1',
    # 'Street Code2',
    # 'Street Code3',
    'Street Name',
    # 'Violation Location',
    # 'Violation Post Code',
    # 'Intersecting Street',
    'Violation Precinct',
    # 'Violation County',
    # 'Vehicle Body Type',
    'Issue Date',
    # 'Date First Observed',
    'Violation Time',
    #'Days Parking In Effect',
    #'From Hours In Effect',
    #'To Hours In Effect',
    # 'Violation Code',
]

file_path = 'all_data_for_one_street.csv'
df = pd.read_csv(file_path, usecols=selected_columns)
df

,Issue Date,Violation Precinct,Violation Time,Street Name
0,06/20/2017,78,1204P,13TH STREET
1,03/09/2017,78,0941A,13TH STREET
2,04/08/2017,78,1109A,13TH STREET
3,09/23/2016,78,1151A,13TH STREET
4,10/18/2016,78,0227P,13TH STREET
...,...,...,...,...
14806,09/11/2023,78,0911A,13TH STREET
14807,09/11/2023,78,0918A,13TH STREET
14808,09/11/2023,78,0918A,13TH STREET
14809,09/14/2023,78,0945A,13TH STREET


In [95]:
df['Violation Precinct'].isna().sum()

0

In [96]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14811 entries, 0 to 14810
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Issue Date          14811 non-null  object
 1   Violation Precinct  14811 non-null  int64 
 2   Violation Time      14811 non-null  object
 3   Street Name         14811 non-null  object
dtypes: int64(1), object(3)
memory usage: 463.0+ KB


In [97]:
df['Violation Time']

0        1204P
1        0941A
2        1109A
3        1151A
4        0227P
         ...  
14806    0911A
14807    0918A
14808    0918A
14809    0945A
14810    0950A
Name: Violation Time, Length: 14811, dtype: object

In [98]:
df_1 = df.copy()

In [99]:
df_1[df_1['Issue Date'].str.contains('AM')]

,Issue Date,Violation Precinct,Violation Time,Street Name
6085,06/13/2019 12:00:00 AM,78,0308A,13TH STREET
6086,06/17/2019 12:00:00 AM,78,0313A,13TH STREET
6087,06/20/2019 12:00:00 AM,78,1245A,13TH STREET
6088,06/20/2019 12:00:00 AM,78,0119P,13TH STREET
6089,06/21/2019 12:00:00 AM,78,0315A,13TH STREET
...,...,...,...,...
7751,06/22/2020 12:00:00 AM,78,1254P,13TH STREET
7752,06/23/2020 12:00:00 AM,78,0420P,13TH STREET
7753,06/24/2020 12:00:00 AM,78,0308P,13TH STREET
7754,06/24/2020 12:00:00 AM,78,0126P,13TH STREET


In [100]:
df_1['Issue Date'] = df_1['Issue Date'].str.split(' ').str[0]

In [101]:
df_1

,Issue Date,Violation Precinct,Violation Time,Street Name
0,06/20/2017,78,1204P,13TH STREET
1,03/09/2017,78,0941A,13TH STREET
2,04/08/2017,78,1109A,13TH STREET
3,09/23/2016,78,1151A,13TH STREET
4,10/18/2016,78,0227P,13TH STREET
...,...,...,...,...
14806,09/11/2023,78,0911A,13TH STREET
14807,09/11/2023,78,0918A,13TH STREET
14808,09/11/2023,78,0918A,13TH STREET
14809,09/14/2023,78,0945A,13TH STREET


In [102]:
def convert_to_24hr_format(time_str):
    try:
        if '.' in time_str:
            return None
        if time_str[-1] == 'P':
            return '{:02d}:{:02d}'.format((int(time_str[:2]) + 12) % 24, int(time_str[2:4]))
        elif time_str[-1] == 'A':
            return '{:02d}:{:02d}'.format(int(time_str[:2]) % 12, int(time_str[2:4]))
        else:
            return None
    except (ValueError, TypeError):
        return None

df_1['Violation Time'] = df_1['Violation Time'].astype(str)
df_1.loc[:, 'Violation Time'] = df_1['Violation Time'].apply(convert_to_24hr_format)
df_1 = df_1.dropna(subset=['Violation Time'])

In [103]:
df_1['Violation Time']

0        00:04
1        09:41
2        11:09
3        11:51
4        14:27
         ...  
14806    09:11
14807    09:18
14808    09:18
14809    09:45
14810    09:50
Name: Violation Time, Length: 14811, dtype: object

In [104]:
df_1['Violation Datetime'] = pd.to_datetime(df_1['Issue Date'] + ' ' + df_1['Violation Time'], format='%m/%d/%Y %H:%M')

In [105]:
df_1['Violation Datetime']

0       2017-06-20 00:04:00
1       2017-03-09 09:41:00
2       2017-04-08 11:09:00
3       2016-09-23 11:51:00
4       2016-10-18 14:27:00
                ...        
14806   2023-09-11 09:11:00
14807   2023-09-11 09:18:00
14808   2023-09-11 09:18:00
14809   2023-09-14 09:45:00
14810   2023-09-08 09:50:00
Name: Violation Datetime, Length: 14811, dtype: datetime64[ns]

In [106]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14811 entries, 0 to 14810
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Issue Date          14811 non-null  object        
 1   Violation Precinct  14811 non-null  int64         
 2   Violation Time      14811 non-null  object        
 3   Street Name         14811 non-null  object        
 4   Violation Datetime  14811 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 578.7+ KB


In [107]:
df_1.sort_values(by=['Violation Datetime'], inplace=True)
df_1

,Issue Date,Violation Precinct,Violation Time,Street Name,Violation Datetime
1836,06/22/2016,78,09:34,13TH STREET,2016-06-22 09:34:00
1844,06/22/2016,78,09:34,13TH STREET,2016-06-22 09:34:00
1840,06/22/2016,78,09:37,13TH STREET,2016-06-22 09:37:00
1852,06/22/2016,78,13:03,13TH STREET,2016-06-22 13:03:00
1849,06/28/2016,78,03:31,13TH STREET,2016-06-28 03:31:00
...,...,...,...,...,...
14646,09/28/2023,78,03:09,13TH STREET,2023-09-28 03:09:00
14647,09/28/2023,78,03:11,13TH STREET,2023-09-28 03:11:00
14648,09/28/2023,78,03:13,13TH STREET,2023-09-28 03:13:00
14649,09/28/2023,78,03:15,13TH STREET,2023-09-28 03:15:00


In [108]:
def calculate_location_violation_rate(df, location, precinct, start_time, end_time):

    one_week_before = start_time - pd.DateOffset(days=5)

    filtered_df = len(df[(df['Street Name'] == location) & (df['Violation Precinct'] == precinct) & (df['Violation Datetime'] >= start_time) & (df['Violation Datetime'] <= end_time)])
    print("filtered_df:", filtered_df)

    total_records = len(df[(df['Violation Datetime'] >= one_week_before) & (df['Violation Datetime'] <= end_time)])
    print("total_records:", total_records)

    if total_records > 0:
        violation_rate = (filtered_df / total_records) * 100
    else:
        violation_rate = 0.0
    return violation_rate

location = '13TH STREET'
location_precinct = 78
violation_datetime = pd.to_datetime('2016-07-11 08:49:00')
estimating_time = pd.to_datetime('2016-07-11 10:59:00')

rate = calculate_location_violation_rate(df_1, location, location_precinct, violation_datetime, estimating_time)
print(f'Location Violation Rate for {location} between {violation_datetime} and {estimating_time}: {rate:.2f}%')

filtered_df: 7
total_records: 31
Location Violation Rate for 13TH STREET between 2016-07-11 08:49:00 and 2016-07-11 10:59:00: 22.58%


In [109]:
def calculate_location_violation_rate(row, df):
    location = row['Street Name']
    precinct = row['Violation Precinct']
    start_time = row['Violation Datetime'] - pd.DateOffset(minutes=30)  # Assuming 1 hour interval
    end_time = row['Violation Datetime'] + pd.DateOffset(minutes=45)  # Assuming 1 hour interval

    one_week_before = start_time - pd.DateOffset(weeks=1)

    filtered_df = len(df[(df['Street Name'] == location) & (df['Violation Precinct'] == precinct) & (df['Violation Datetime'] >= start_time) & (df['Violation Datetime'] <= end_time)])

    total_records = len(df[(df['Violation Datetime'] >= one_week_before) & (df['Violation Datetime'] <= end_time)])

    if total_records == 0:
        return 0.0

    if total_records > 0:
        violation_rate = (filtered_df / total_records) * 100
    else:
        violation_rate = 0.0

    return violation_rate

df_1['Violation_Rate'] = df_1.apply(calculate_location_violation_rate, df=df_1, axis=1)

In [110]:
df_1['Violation_Rate'].mean()

11.700704008823255

In [111]:
df_1 = df_1.iloc[8:]
df_1

,Issue Date,Violation Precinct,Violation Time,Street Name,Violation Datetime,Violation_Rate
1837,06/29/2016,78,16:23,13TH STREET,2016-06-29 16:23:00,20.000000
1845,06/29/2016,78,20:40,13TH STREET,2016-06-29 20:40:00,16.666667
1838,06/30/2016,78,08:52,13TH STREET,2016-06-30 08:52:00,50.000000
1856,06/30/2016,78,08:53,13TH STREET,2016-06-30 08:53:00,50.000000
1855,06/30/2016,78,08:55,13TH STREET,2016-06-30 08:55:00,50.000000
...,...,...,...,...,...,...
14646,09/28/2023,78,03:09,13TH STREET,2023-09-28 03:09:00,9.803922
14647,09/28/2023,78,03:11,13TH STREET,2023-09-28 03:11:00,9.803922
14648,09/28/2023,78,03:13,13TH STREET,2023-09-28 03:13:00,9.803922
14649,09/28/2023,78,03:15,13TH STREET,2023-09-28 03:15:00,9.803922


In [112]:
df_1['Violation_Rate'].max()

100.0

In [113]:
# def scale_between_0_and_1(value, min_value, max_value):
#     """
#     Scale a value to be between 0 and 1.
#
#     :param value: The value to be scaled.
#     :param min_value: The minimum value in the range of your data.
#     :param max_value: The maximum value in the range of your data.
#     :return: Scaled value between 0 and 1.
#     """
#     if max_value == min_value:
#         return 0  # Avoid division by zero if all values are the same
#
#     return (value - min_value) / (max_value - min_value)

# df_1['Violation_Rate'] = df_1['Violation Rate'].apply(lambda x: scale_between_0_and_1(x, df_1['Violation Rate'].min(), df_1['Violation Rate'].max()))
# df_1['Violation_Rate']

In [114]:
df_1

,Issue Date,Violation Precinct,Violation Time,Street Name,Violation Datetime,Violation_Rate
1837,06/29/2016,78,16:23,13TH STREET,2016-06-29 16:23:00,20.000000
1845,06/29/2016,78,20:40,13TH STREET,2016-06-29 20:40:00,16.666667
1838,06/30/2016,78,08:52,13TH STREET,2016-06-30 08:52:00,50.000000
1856,06/30/2016,78,08:53,13TH STREET,2016-06-30 08:53:00,50.000000
1855,06/30/2016,78,08:55,13TH STREET,2016-06-30 08:55:00,50.000000
...,...,...,...,...,...,...
14646,09/28/2023,78,03:09,13TH STREET,2023-09-28 03:09:00,9.803922
14647,09/28/2023,78,03:11,13TH STREET,2023-09-28 03:11:00,9.803922
14648,09/28/2023,78,03:13,13TH STREET,2023-09-28 03:13:00,9.803922
14649,09/28/2023,78,03:15,13TH STREET,2023-09-28 03:15:00,9.803922


In [115]:
# df_1.to_csv('analyzed_data_for_one_street.csv', index=False)